## Think Like An Adversary, Protect Oski

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import laplace
from IPython.display import Markdown as md

### Brown Bears, Salmon and Oski

Suppose we are studying the impact of dietary habits of the brown bear and their implications on the loss of fur of our salmon eating friends. The database contains a list of bears, all of whom are losing their fur.  They are balding.  No self-respecting bear wants such a sensitive fact made public. Disclosure of a bear's membership in this database means breaching the privacy of the bear's senstive attribute: they are going bald.

Consider the facts around the Balding Bears database.

- When the salmon are running large, talented and strong brown bears can catch in excess of 30-50 salmon per day
- Lesser talented, "normal" bears tend to collect and eat 10 - 20 salmon per day


Suppose we have a database of ten bears, all somewhat average brown bears, who are all balding.  Initially we have 10 bears in the database.

1. Avalanche (Kutztown)
2. Bananas (U of Maine)
3. Benny (Morgan State)
4. Boomer (Lake Forest)
5. Grizz (Oakland)
6. Kody (Cascadia)
7. Monte (U Montana)
8. Nanook (Bowdoin)
9. Ranger (Drew)
10. Scott Highlander (UC Riverside)

On a particular day, July 30, when the salmon are running the database records the following number of fish captured by this group.

[10, 13, 14, 12, 18, 14, 18, 17, 16, 12]

**Note: the eating habits of this entire group falls within a reasonable range of "normal" brown bear daily catches!**

It is well known fact that **Oski the Cal Bear** is a giant among bears. What if we have auxillary information about Oski?  Namely, true to his school population he is talented, competitive, clever, and innovative and ee may be able to catch **40-50** salmon a day when they are running!

**What if we needed to protect the privacy of outlier bear we well, one such as Oski?**
What if the database, in general had to protect a wider range of dietary habits of our salmon eaters? Not only is the protection important but perhaps the outliers hold the key to uncovering a cure for the balding bears?


![Oski](./images/oski.png)

Below are the numbers of the eating habits of our bears on July 30th, including an addition, our salmon catching star Oski, the Cal mascot.

[10, 13, 14, 12, 18, 14, 18, 17, 16, 12, 48]

In [13]:
#Create a dataframe of the balding bears
baldingBears = pd.DataFrame({'Bears': ['Avalanche', 'Bananas', 'Benny', 'Boomer', 'Grizz', 'Kody', 'Monte', 'Nanook', 'Ranger', 'Scott Highlander', 
                                       'Oski'],'School': ['Kutztown', 'U of Maine', 'Morgan State', 'Lake Forest', 'Oakland', 'Cascadia', 'U Montana', 
                                       'Bowdoin', 'Drew', 'UC Riverside','Berkeley'], 'Salmon':[10, 13, 14, 12, 18, 14, 18, 17, 16, 12, 48]})


In [11]:
# add 1 to each index
baldingBears.index = baldingBears.index + 1
baldingBears

,Bears,School,Salmon
1,Avalanche,Kutztown,10
2,Bananas,U of Maine,13
3,Benny,Morgan State,14
4,Boomer,Lake Forest,12
5,Grizz,Oakland,18
6,Kody,Cascadia,14
7,Monte,U Montana,18
8,Nanook,Bowdoin,17
9,Ranger,Drew,16
10,Scott Highlander,UC Riverside,12


**Let's assume we have a rival to our Cal Bear that wants to bring Oski down by exposing his balding woes.**  Recall that the true sum of the total salmon eaten in the database without Oski in it is $144$ and when we add him in the true sum jumps to $192$.  Since these are "adjacent databases", namely databases that differ by $1$ record, the substantial jump in the sum, couple with prior knowledge that Oski is a very big eater, an outlier, a jump would be a signal to an attacker, statistically speaking that could expose Oski's membership.

In [14]:
#Create a dataframe of the balding bears
baldingBears_no_Oski = pd.DataFrame({'Bears': ['Avalanche', 'Bananas', 'Benny', 'Boomer', 'Grizz', 'Kody', 'Monte', 'Nanook', 'Ranger', 'Scott Highlander'],
                                    'School': ['Kutztown', 'U of Maine', 'Morgan State', 'Lake Forest', 'Oakland', 'Cascadia', 'U Montana', 
                                       'Bowdoin', 'Drew', 'UC Riverside'], 
                                    'Salmon':[10, 13, 14, 12, 18, 14, 18, 17, 16, 12]})
baldingBears_no_Oski

,Bears,School,Salmon
0,Avalanche,Kutztown,10
1,Bananas,U of Maine,13
2,Benny,Morgan State,14
3,Boomer,Lake Forest,12
4,Grizz,Oakland,18
5,Kody,Cascadia,14
6,Monte,U Montana,18
7,Nanook,Bowdoin,17
8,Ranger,Drew,16
9,Scott Highlander,UC Riverside,12


#### 3.1 Threat Framework

Suppose we have an attacker who has heard rumors and observed fur left behind by Oski that leads him believe that Oski has a 50-50 chance of becoming bald. This is **attacker's prior** belief, is that it is equally likely that Oski is in the database and not in the database.

Just a few probability definitions, we promise.

Let:

- **Oski** be the event that Oski is in the database
- **noOski** be the event that Oksi is not in the database

The Odd's Ratio here is:

$$\frac{P(noOksi)}{P(Oski)}$$

- This ratio is known in probability as the **"Odds Ratio"**.
- The **Prior Odd's Ratio** is the Odd's Ratio before the database query outcomes are observed.
- The **Posterior Odd's Ratio** is the Odd's Ratio after, or conditioned on the observing the outcomes. 

While we won't show you the derivation here, but it can be proven mathematically that the **"Prior Odds Ratio"** multiplied another ratio, the  **"Likelihood Ratio"** is equivalent to the Odds Ratio after observing the statistical behavior of the outcome for many runs of a query approximates, namely, the **"Posterier Odds Ratio"**.

**How does this help explain our threat model?**

In our threat model, an attacker has a preconditioned notion of Oski's odds of being a balding bear. Then the attacker observes many runs of a query on the database and observes the results.  These results present an approximation of the "Likelihood Ratio".  Let's say the Likelihood Ratio moves the attacker from his initial belief to a stronger belief that Oski is more likely to be in the database.  This is the Posterior Odd's Ratio. Practically speaking, at some point if he gains enough information, those post observation odds for the attacker will breach a practical threshold which, in his mind, is a decision point.  At that decision point, (when the odds are high enough), the attacker will conclude Oksi is in the database. Oski's privacy will be violated.

In our example, let's assume that if the attacker's Odd's Ratio moves to greater than 2.  In statsitical vernacular, the Attacker has a Posterior Odds Ratio > 2, he concludes that Oksi is in the database and for this attacker he is willing to tell the world of his findings and Oski's privacy will be breached.

 

### 3.2 Simple Example With Oski and an Attacker

Let's illustrate our threat framework with a simple example. Here are the basic components.

Recall from our explanation in the last section that:

- Odds Ratio (Posterior) = Odds Ratio (Prior) x Likelihood Ratio 

- We know an attacker can simulate the Likelihood Ratio through many runs of queries on the database.  (The Likelihood Ratio can also be calculated mathematically for any probability distribution.)

- We know the Prior Odds Ratio because we know or can guess the mindset of our attacker, namely the threshold of the Posterior Odds Ratio which once exceeded the attacker will conclude Oski is in the database.

- Here we assume that an Attacker will conclude Oski is in the database if his Posterior Odds Ratio exceeds 2. 

- Let **O be the outcome that the Differentially Privacy mechanism returns SUM >= 164**

- Specifically we are looking for the ratio of the probability that SUM > 164 without Oski in the database over the probability of SUM > 164 when Oski is in the database.

- The prior belief by our attacker is that the odds of Oski being in the database are 50-50.


In [21]:
def laplaceMechanism(x, epsilon, sensitivity):
    noisyX =  np.random.laplace(x, 2*sensitivity/epsilon, 1)[0]
    return noisyX

def ANON_SUM(col, epsilon, u, l):
    trueSum = np.sum(col)
    sensitivity = abs(u-l)
    noisySum = laplaceMechanism(trueSum, epsilon,sensitivity)
    return noisySum

In [22]:
# Set Differential Privacy Parameters for Anonymous Sum
# Run the sum query 10,000 times 
# Privacy leakage is relatively high, sensitivity is low and thus even more revealing
u = max(baldingBears_no_Oski['Salmon'])
l = min(baldingBears_no_Oski['Salmon'])
epsilon = 4
col = baldingBears_no_Oski['Salmon']
noisySums = []
for i in range(10000):
    noisySum = ANON_SUM(col, epsilon, u, l)  
    noisySums.append(noisySum)

In [27]:
a = sum(i > 164 for i in noisySums)

In [28]:
md("Notice with these parameter settings, the number observations with sum >= 164 is {}!".format(a))

Notice with these parameter settings, the number observations with sum >= 164 is 35!

Now add Oski to the database and run the same queries again.

In [29]:
col = baldingBears['Salmon']
# Run the sum query 10,000 times 
# Privacy leakage is relatively high, sensitivy is low and thus even more revealing
noisySums = []
for i in range(10000):
    noisySum = ANON_SUM(col, epsilon, u, l)  
    noisySums.append(noisySum)

In [31]:
b = sum(i > 164 for i in noisySums)
md("Notice with these parameter settings, the number observations with sum >= 164 is {}!".format(b))

Notice with these parameter settings, the number observations with sum >= 164 is 9998!

In [35]:
md("With these parameter settings, for the event O, the Likelihood Ratio is {} !".format(round(b/a, 2)))
md("Because {} is clearly higher than 2, the attacker will conclude Oski is in the database!".format(round(b/a, 2)))

Because 285.66 is clearly higher than 2, the attacker will conclude Oski is in the database!

**Make it harder for our attacker to breach his confidence threshold by setting our $\epsilon$ privacy parameter lower, set the senstivity $\Delta$ higher tuning for less privacay leakage.**

In [36]:
# Set Differential Privacy Parameters for Anonymous Sum
u = 40 # User input
l = 10 # User input
epsilon = 1 #low epsilon, less provacy leakage
col = baldingBears_no_Oski['Salmon']
# Run the sum query 10,000 times 
# Privacy leakage is relatively high, sensitivy is low and thus even more revealing
noisySums = []
for i in range(10000):
    noisySum = ANON_SUM(col, epsilon, u, l)  
    noisySums.append(noisySum)

In [37]:
a = sum(i > 164 for i in noisySums)
md("Notice with these parameter settings, the number observations with sum >= 164 is {}!".format(a))

Notice with these parameter settings, the number observations with sum >= 164 is 3608!

In [38]:
# Set Differential Privacy Parameters for Anonymous Sum
col = baldingBears['Salmon']
# Run the sum query 10,000 times 
# Privacy leakage is relatively high, sensitivy is low and thus even more revealing
noisySums = []
for i in range(10000):
    noisySum = ANON_SUM(col, epsilon, u, l)  
    noisySums.append(noisySum)

In [40]:
b = sum(i > 164 for i in noisySums)
md("Notice with these parameter settings, the number observations with sum >= 164 is {}!".format(b))

Notice with these parameter settings, the number observations with sum >= 164 is 6843!

In [48]:
md("With these parameter settings, for the event O, the Likelihood Ratio is {} !".format(round(b/a, 2)))
md("Because {} is lower than 2, the attacker will not be able to conclude Oski is in the database!".format(round(b/a, 2)))
md("Oski is protect4ed in the threat model, by the parameters epsilon = {} and sensitivity = {}!".format(epsilon, u-l))

Oski is protect4ed in the threat model, by the parameters epsilon = 1 and sensitivity = 30!

***

### 3.3 Deeper Dive into a Bayesian Threat Model (Optional)

Recall the standard definition of a Bayesian inference. 
Let 

- O = Outcome O, 

- noOksi mean Oski is not in the database

- Oski mean Oski is in the database.  

We know the following.

$$P(noOski | O) = \Bigg [\frac{P(noOski)*P(O | noOski)}{P(O)}\Bigg ]$$

$$P(Oski | O) = \Bigg [\frac{P(Oski)*P(O | Oski)}{P(O)}\Bigg ]$$

We also know that the Odds Ratio is the following.

$$\frac{P(noOski)}{P(Oski)}$$

Here the Odds Ratio is the "Prior" the belief that the attacker has prior to seeing any of the data in the database. In the example above we assumed the Attacker had good reasons to believe Oksi we in the database and thought the odds of Oksi going bald was 50% thus the Odds Ratio was 1.

Take the ratio of the first two equations, placing the second in the numerator and the first in the demoninator. (It does not really matter).  We get the following.

$$ \frac{P(Oski | O)}{P(noOski | O)} = \Bigg[ \frac{P(Oski) * P(O|Oski)}{P(noOski) * P(O | noOski)}\Bigg ]= $$ 




$$ \Bigg[ \frac{P(Oski)}{P(noOski)} \Bigg ] * \Bigg [\frac{P(O | Oski)}{P(O | noOski}\Bigg ] $$

Simply, 

$$ \frac{P(Oski | O)}{P(noOski | O)} = \Bigg[ \frac{P(Oski)}{P(noOski)} \Bigg ] * \Bigg [\frac{P(O | Oski)}{P(O | noOski}\Bigg ] $$

**This is very simply the Odds Ratio after observing the database* equal to the Odds Ratio prior to observing the database (the Adversary's prior belief, multiplied by the **Likelhood Ratio of the outcome, conditioned on whether or not Oski is in the database!**

To summarize: 

- First term on the right hand side is assumed about the Attacker, (Prior Odds Ratio)
- Second term is observed through statistical runs of the query (or computed mathematically) 
- Product on the right hand side yeilds the Attackers belief after observing the database. 

In the previous simple example above we set the threshold for for a successful attack at 2, but this threshold could be set for a practical privacy level that is use case dependent.  